In [2]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 30 08:19:40 2019
https://www.kaggle.com/takuok/bidirectional-lstm-and-attention-lb-0-043
@author: caridza
"""
import os 
import sys
import pandas as pd 
import pymagnitude
import re
import string 
import numpy as np
import nltk
from sklearn import preprocessing , model_selection, metrics
from sklearn.model_selection import StratifiedShuffleSplit
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk import regexp_tokenize
from nltk.tokenize import word_tokenize , sent_tokenize

#MUST IMPORT ALL FROM EITHER KERAS or Tensorflow.python.keras. YOU CANNOT MIX
import keras 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras_self_attention import SeqSelfAttention 

import gensim
from imblearn.over_sampling import SMOTE  # or: import RandomOverSampler
from imblearn.pipeline import Pipeline as imbPipeline


In [3]:
#stopwords, stemmer
datapath = "//home//nlpsomnath//NegNews/zackc//Misc Notebooks//Python Misc//Data//SentDF.pickle"
stemmer = SnowballStemmer('english')
exclude = set(string.punctuation)
stopwords = stopwords.words('english')
newStopWords = ['.','?','%','Google','Wells Fargo','Donald Trump','Charles Schwab','Morgan Stanley','Credit Suisse','Reuters','Bank of America','Guggenheim','Deutsch Bank','Goldman Sachs','Facebook','Fifth Third Bank','New York','Washington','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','January','February','March','April','May','June','July','August','September','October','November','December']
stopwords.extend(newStopWords)
stop_list=set(stopwords)
wv=pymagnitude.Magnitude('//home//nlpsomnath//NegNews//rebase//NN_Kafka//sourcefiles//GoogleNews-vectors-negative300.magnitude')

#inputs requiring path specification
max_features = 100000 #maximum number of words to consider in analysis 
EMBEDDING_DIM = 300 #NOTE: becuase we use pymag this must be set to 300 
stop_list = stop_list
stemmer = stemmer
target = 'MandA'


In [11]:
#generate unbalanced holdout and balanecd training data for model training
Data = pd.read_pickle(datapath)


In [7]:

sss = StratifiedShuffleSplit(n_splits=2, test_size=0.1, random_state=0)
sss.get_n_splits(Data.index.values, Data[target])
indicies  = []
for train_index ,test_index in sss.split(Data.index,Data[target]):
    indicies.extend(test_index)
testrows=indicies
holdout= Data.iloc[testrows,:]  


#create balanced dataframe for modeling
data = Data.loc[set(Data.index)-set(holdout.index)]
balanced_data = upsample_rare(data,target)


#transform original Df into sequence tokenized data and split into test and train. get list of words in vocab , and maxlen of sequence
xtr,ytr,xte,yte,word_index, maxlen,words = make_df(Data,      #path to sentence level data 
                                             max_features,  #maximum number of words to consider in model 
                                             EMBEDDING_DIM, #number of dims form embedding vector to use(if pymag used this must be 300)
                                             stop_list,
                                             stemmer,
                                             target)


#generate embedding vector using pymagnitude 
nb_words = min(max_features, len(word_index)+1) #total features to consider, min of the max feats vs total feats 
embedding_vector =  make_embeddings_pymag(wv, max_features,words, word_index, EMBEDDING_DIM, nb_words)

#compile sequence model layers
model=keras.models.Sequential()
#add embedding layer
model.add(keras.layers.Embedding(input_dim = nb_words #max_features
                                 , output_dim= EMBEDDING_DIM #embedding size 
                                 , weights = [embedding_vector]
                                 , mask_zero=False
                                 #, trainable=False
                                 , input_length = maxlen #the max sequence length, this is the length that all sequences will be padded to (so all sequences will have same length)
                                 ))
#add bilstm layer with dropout 
#model.add(keras.layers.Bidirectional(keras.layers.LSTM(units = 300, return_sequences=True,dropout=.3,recurrent_dropout=.3)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(units = 300, 
                                                       return_sequences=True,
                                                       dropout=.6,
                                                       recurrent_dropout=.6,
                                                       recurrent_regularizer=keras.regularizers.L1L2(l1=0.0, l2=0.01),
                                                       kernel_regularizer = keras.regularizers.L1L2(l1=0,l2=.01),
                                                       #bias_regularizer = keras.regularizers.L1L2(l1=0,l2=.01),
                                                       
                                                       )))

#add self attention layer 
#model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(SeqSelfAttention(attention_width=15,
                        attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                        attention_activation=None,
                        kernel_regularizer=keras.regularizers.L1L2(1e-6),
                        bias_regularizer=keras.regularizers.l1(1e-4),
                        use_attention_bias=False,    
                        attention_regularizer_weight=1e-4,
                        name='Attention',))

#flatten layers
model.add(keras.layers.Flatten())
#add output layer
model.add(keras.layers.Dense(units =1,activation='sigmoid'))

#compile model 
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics =['binary_accuracy'] )
model.summary()


#specify callbacks for keras model 
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
ckpt = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor="val_loss", mode="min", patience=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 79, 300)           14225700  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 79, 600)           1442400   
_________________________________________________________________
seq_self_attention_1 (SeqSel (None, 79, 600)           38465     
_________________________________________________________________
flatten_1 (Flatten)          (None, 47400)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 47401     
Total params: 15,753,966
Trainable params: 15,753,966
Non-trainable params: 0
_________________________________________________________________


In [9]:

#fit model 
model_out = model.fit(xtr, ytr, batch_size=200, epochs=15, validation_split=0.2)#, callbacks=[ckpt, early])

#evaluate model fit on test data
#y_preds = model.predict_classes(x=xte)
y_preds = model.predict_classes(xte)
matrix = metrics.confusion_matrix(yte, y_preds)
scores = model.evaluate(xte, yte, verbose=0)
print(matrix)

#save model to disk 
# serialize model to YAML
#model_yaml = model.to_yaml()
#with open("model_RegRef.yaml", "w") as yaml_file:
#    yaml_file.write(model_yaml)

# serialize weights to HDF5
#model.save_weights("modelWeights_Regref.h5")
#print("Saved model to disk")
#loading model for scoring new data 
# # load YAML and create model
# yaml_file = open('model.yaml', 'r')
# loaded_model_yaml = yaml_file.read()
# yaml_file.close()
# loaded_model = model_from_yaml(loaded_model_yaml)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")

Train on 184819 samples, validate on 46205 samples
Epoch 1/15
184819/184819 [==============================] - 1527s 8ms/step - loss: 0.3560 - binary_accuracy: 0.8635 - val_loss: 0.3853 - val_binary_accuracy: 0.8358
Epoch 2/15
184819/184819 [==============================] - 1299s 7ms/step - loss: 0.3168 - binary_accuracy: 0.8826 - val_loss: 0.4885 - val_binary_accuracy: 0.7820
Epoch 3/15
184819/184819 [==============================] - 1295s 7ms/step - loss: 0.2907 - binary_accuracy: 0.8944 - val_loss: 0.6703 - val_binary_accuracy: 0.7179
Epoch 4/15
184819/184819 [==============================] - 1294s 7ms/step - loss: 0.2702 - binary_accuracy: 0.9046 - val_loss: 0.6439 - val_binary_accuracy: 0.7402
Epoch 5/15
184819/184819 [==============================] - 1292s 7ms/step - loss: 0.2522 - binary_accuracy: 0.9125 - val_loss: 0.6698 - val_binary_accuracy: 0.7338
Epoch 6/15
184819/184819 [==============================] - 1291s 7ms/step - loss: 0.2341 - binary_accuracy: 0.9195 - val_lo

In [9]:
#check confusion matrix on validation set 
y_preds = model.predict_classes(xte)
matrix = metrics.confusion_matrix(yte, y_preds)
print(matrix)

[[40464  9042]
 [  257  1413]]


In [4]:

##############################################################################################
##############################################################################################
#######################################FUNCTIONS##############################################
##############################################################################################
##############################################################################################

#FUNCTION TO CLEAN ORIGINAL TEXT FOR MODEL INPUT
def orig_text_clean(data,target='LegalAction',txtfeild='origtext',maplabelvars=['source']
                    ,stopwords=['the','is','a','i','are','it'],stemmer=None,score_new=False):
    trainDF = pd.DataFrame()
    trainDF['text'] = data[txtfeild].apply(lambda x: remove_punctuation(x))
    trainDF['text'] = trainDF['text'].apply(lambda x: remove_nonchars(x))
    trainDF['text'] = trainDF['text'].apply(lambda x: remove_stop(x,stopwords=stopwords))
    trainDF[txtfeild] = trainDF['text'].apply(lambda x: stem_words(x,stemmer=stemmer))
    
    if score_new==False:
        trainDF['label'] = data[target]
        le = preprocessing.LabelEncoder() 
        le.fit(trainDF['label'])
        trainDF['label_id'] =le.transform(trainDF['label'])
            
    trainDF=trainDF.reset_index(drop=True)
    return trainDF 

#preprocess data 
#func to remove punc from string and return string
def remove_punctuation(text,excluded_punct={'+', ':', '[', '^', '"', '|', '{', '@', '=', ')', '%', '#', '`', '}', "'", '(', ',', '!', '*', '_', '>', '?', '&', '-', '~', '\\', '<', '/', '.', ']', ';', '$'}):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in excluded_punct])

#func to remove stop words 
def remove_stop(text,stopwords=['the','is','a','i','are','it']):
    return ' '.join([word for word in text.split(' ') if word.lower() not in stopwords])

#func to stem words 
def stem_words(text,stemmer=None):
    return ' '.join([stemmer.stem(word) for word in text.split(' ')])

#remove non alpha characters from text 
def remove_nonchars(text):
    return ' '.join([re.sub('[^A-Za-z|^\$|^\.]+', ' ', word) for word in text.split(' ') if (word.isalnum() and len(word)>2)])






#####MODEL DATA PREPROCESSING FUNCTION 
def make_df(datapath , max_features,EMBEDDING_DIM,stop_list,stemmer,target):
    '''
    Inputs: 
    ##### datapath = path to dataframe of sentences and label ids 
    ##### max_features = total number of features from text to consider when buildling embedding layer
    ##### EMBEDDING_DIM = total number of dimensions from embedding we want to use in embedding layer (Note: if using pymag you must use 300)
    ##### stop_lsit = list of stopwords to use for preprocessing 
    ##### stemmer = stemming class to use to stem words in sentences 
    ##### target = name of the target feild you are trying to predict in the dataset 
    PROCESS STEPS: 
    ##### 1. toeknize text
    ##### 2. Convert text to sequences
    ##### 3. Pad Sequences 
    ##### 4. Split into Test and Train
    ##### 5. Return X_train , X_Test, Y, WordIndex 
    '''
    
    #load data
    if isinstance(datapath,object): 
        data = datapath 
    else:
        data = pd.read_pickle(datapath)
    
    #clean original sentence level text
    trainDF = orig_text_clean(data,target=target , txtfeild='Sentence',stopwords=stop_list,stemmer=stemmer) 
    
    #specify X and Y (these standard columns are created in orig_text_clean)
    X = trainDF['Sentence']
    Y = trainDF['label_id']
    
    #generate list of unique words based on total words specified to consider
    sentences = []
    lines = X.values.tolist()
    lines = [word_tokenize(sent) for sent in lines]
    model=gensim.models.Word2Vec(sentences=lines, size=EMBEDDING_DIM,window=5,workers=4,min_count=1)
    words = list(model.wv.vocab)
    
    #model process 
    #fit tokenizer to words and turn to sequences 
    tokenizer_obj = Tokenizer()
    tokenizer_obj.fit_on_texts(X)
    sequences = tokenizer_obj.texts_to_sequences(X)
    
    #define max length for padding and total vocab size
    max_length = max([len(s.split()) for s in X]) 
    vocab_size = len(tokenizer_obj.word_index)+1
    
    #pad sequences 
    word_index = tokenizer_obj.word_index
    review_pad = pad_sequences(sequences,maxlen=max_length)
    label = Y.values
    
    #split data 
    x_train,x_test,y_train,y_test = model_selection.train_test_split(review_pad, label,shuffle=True, stratify=Y,test_size=.3, random_state=10)
    sm = SMOTE(random_state=2)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    
    return x_train, y_train, x_test, y_test, word_index, max_length , words




#EMBEDDING GENERATION FUNCTIONS:FUNCTIONS TO GENERATE EMBEDDINGS FROM PRETRAINED EMBEDDINGS FOR EMBEDDINGS LAYER IN NN 
#glove vectors
def make_glovevec(glovepath, max_features, embed_size, word_index, veclen=300):
    embeddings_index = {}
    f = open(glovepath)
    for line in f:
        values = line.split()
        word = ' '.join(values[:-300])
        coefs = np.asarray(values[-300:], dtype='float32')
        embeddings_index[word] = coefs.reshape(-1)
    f.close()

    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

#pymagnitude vectors 
def make_embeddings_pymag(wv, max_features,words, word_index, embed_size, nb_words):
    #embeddings using pymagnitude 
    embeddings_index = {}
    for w in words:
        word =w
        coefs = wv.query(word) #np.asarray(values[1:])
        embeddings_index[word]=coefs

    embedding_matrix = np.zeros((nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix


###FUNCTION TO GENERATE BI-DIRECTIONAL LSTM WITH EMBEDDING AND ATTENTION
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

def BidLstm(maxlen, max_features, embed_size, embedding_matrix):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, #vocabuarly size 
                  embed_size,  #total size of output embedding dimensions
                  weights=[embedding_matrix], #matrix associated with pre-trained embedding vector values 
                  trainable=False #trainable infers the matrix weights for the embeddings will change at each iteration of training to reflect the context the words are used in during training, as opposed to there general context 
                  )(inp)
    x = Bidirectional(LSTM(300, return_sequences=True, dropout=0.25,
                           recurrent_dropout=0.25))(x)
    x = Attention(maxlen)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)

    return model

#FUNCTIONS TO PLOT OUTPUT FROM KERAS MODEL BUILD 
 #plot model training history 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
def plot_history(history,metric='acc'):
    #https://www.kaggle.com/danbrice/keras-plot-history-full-report-and-grid-search
    acc = history.history[metric]
    val_acc = history.history['val_'+metric]
    loss = history.history['loss']
    val_loss = history.history['val_'+metric]
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title='Normalized confusion matrix'
    else:
        title='Confusion matrix'

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
##################################################################################
###############ATTENTION CLASS REQUIRED TO USE ATTENTINO IN LSTM##################
##################################################################################
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
    
    
    
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 31 13:17:24 2019

@author: caridza
"""

def plot_scorers(gs_cv_results,hyperparm):
    results = gs_cv_results
    plt.figure(figsize=(13, 13))
    plt.title("GridSearchCV evaluating using multiple scorers simultaneously",
              fontsize=16)
    
    plt.xlabel("min_samples_split")
    plt.ylabel("Score")
    
    ax = plt.gca()
    ax.set_xlim(0, 1)
    ax.set_ylim(0.8, 1)
    
    # Get the regular numpy array from the MaskedArray
    X_axis = np.array(results[hyperparm].data, dtype=float)
    
    for scorer, color in zip(sorted(scoring), ['b', 'g', 'r', 'c', 'm', 'y', 'k']):
        print(scorer)
        for sample, style in (('train', '--'), ('test', '-')):
            sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
            sample_score_std = results['std_%s_%s' % (sample, scorer)]
            ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                            sample_score_mean + sample_score_std,
                            alpha=0.1 if sample == 'test' else 0, color=color)
            ax.plot(X_axis, sample_score_mean, style, color=color,
                    alpha=1 if sample == 'test' else 0.7,
                    label="%s (%s)" % (scorer, sample))
    
        best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
        best_score = results['mean_test_%s' % scorer][best_index]
    
        # Plot a dotted vertical line at the best score for that scorer marked by x
        ax.plot([X_axis[best_index], ] * 2, [0, best_score],
                linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)
    
        # Annotate the best score for that scorer
        ax.annotate("%0.2f" % best_score,
                    (X_axis[best_index], best_score + 0.005))
    
    plt.legend(loc="best")
    plt.grid('off')
    plt.show()
    
    list(results)
    

def upsample_rare(data, Rare_ColStr):
    '''
    Input: {'data':'dataframe with rare event','Rare_ColStr':'name of the column containing the rare target event we want to oversample'
    Output: Dataframe of equal proportions of target event to be used only in training (MUST SPLIT TEST DATA SET BEFORE RUNNING THIS)

    '''
    negobs = data[data[Rare_ColStr]==False]
    posobs = data[data[Rare_ColStr]==True]

    #create a list of dataframes, where each data frame contains a replica of all postive observations which we can use for oversampling 
    rep_1 =[posobs for x in range(negobs.shape[0]//posobs.shape[0] )]
    keep_1s = pd.concat(rep_1, axis=0)
    train_dat = pd.concat([keep_1s,negobs],axis=0)
    return(train_dat)